In [25]:
# Notebook for analytics on the result of the republic referendum

### Libraries

In [26]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Image

In [27]:
filepath = '1999_referenda_output/republic_referendum_by_electorate_by_polling_place.csv'
df_results = pd.read_csv(
    filepath
)

display(df_results.head(3))

index state electorate      polling_place    polling_place_raw yes_or_no  \
0      0    SA   Adelaide           Adelaide  Adelaide (Adelaide)       Yes   
1      1    SA   Adelaide      Adelaide East        Adelaide East       Yes   
2      2    SA   Adelaide  Adelaide Hospital    Adelaide Hospital       Yes   

   yes_n   yes_p  no_n    no_p  formal_n  formal_p  informal_n  informal_p  \
0    282  0.6144   177  0.3856       459    0.9871           6      0.0129   
1    465  0.6700   229  0.3300       694    0.9914           6      0.0086   
2    187  0.6172   116  0.3828       303    0.9806           6      0.0194   

   total_n  total_p  
0      465   0.0057  
1      700   0.0086  
2      309   0.0038

In [28]:
filepath = '1999_referenda_output/republic_referendum_by_polling_place.csv'
df_results_by_pp = pd.read_csv(
    filepath
)

display(df_results_by_pp.head(3))

state  polling_place  yes_n  no_n  formal_n  informal_n  total_n   yes_p  \
0   ACT        Ainslie   1372   500      1872          24     1896  0.7329   
1   ACT  Ainslie North   1608   749      2357          29     2386  0.6822   
2   ACT         Aranda   2200   787      2987          21     3008  0.7365   

     no_p  formal_p  informal_p  
0  0.2671    0.9873      0.0127  
1  0.3178    0.9878      0.0122  
2  0.2635    0.9930      0.0070

In [29]:
filepath = '1999_referenda_output/polling_places_geocoded.csv'
df_pp = pd.read_csv(
    filepath
)

display(df_pp.head(3))

state      polling_place                 premises  \
0   ACT           Bonython  Bonython Primary School   
1   ACT            Calwell      Calwell High School   
2   ACT  Canberra Hospital    The Canberra Hospital   

                     address    suburb  postcode wheelchair_access  \
0                 Hurtle Ave  BONYTHON    2905.0                 F   
1                 Casey Cres   CALWELL    2905.0                 F   
2  Blding 2 Level 3 Yamba Dr    GARRAN    2605.0                 F   

          match_source                            match_type  latitude  \
0  2007 Polling Places  Match 01 - state, premises, postcode  -35.4318   
1  2007 Polling Places  Match 01 - state, premises, postcode  -35.4406   
2  2007 Polling Places  Match 01 - state, premises, postcode  -35.3453   

   longitude  
0    149.083  
1    149.116  
2    149.100

### Result by state/territory

* The ACT comfortably voted in favour, and was the only jurisdiction to do so
* Victoria was very narrowly opposed
* Queensland was almost 2 to 1 against

In [30]:
r = df_results[['state','yes_n','formal_n']].groupby('state').sum()
r['yes_p'] = round(r['yes_n']/r['formal_n'],4)
display(r.sort_values(['yes_p'],ascending=False))

yes_n  formal_n   yes_p
state                           
ACT     127211    201061  0.6327
VIC    1489536   2988674  0.4984
NT       44391     91028  0.4877
NSW    1817380   3913942  0.4643
SA      425869    977444  0.4357
WA      458306   1104826  0.4148
TAS     126271    312784  0.4037
QLD     784060   2094052  0.3744

### The inner cities were the most strongly in favour

The top 5 seats by proportion in favour represent two inner Melbourne and two inner Sydney seats, as well as one of the two Canberra seats

In [31]:
r = df_results[['electorate','yes_n','formal_n']].groupby('electorate').sum()
r['yes_p'] = round(r['yes_n']/r['formal_n'],4)
display(r.sort_values(['yes_p'],ascending=False).head(5))

yes_n  formal_n   yes_p
electorate                              
Melbourne        59994     84598  0.7092
Sydney           56921     83894  0.6785
Melbourne Ports  51520     78183  0.6590
Grayndler        51774     79929  0.6477
Fraser           64636    100266  0.6446

### Rural Australia was the most strongly opposed

The top 5 seats by proportion opposed are all large rural seats, four in Queensland and one (Gwydir) in NSW.

In [32]:
r = df_results[['electorate','yes_n','formal_n']].groupby('electorate').sum()
r['yes_p'] = round(r['yes_n']/r['formal_n'],4)
display(r.sort_values(['yes_p'],ascending=True).head(5))

yes_n  formal_n   yes_p
electorate                         
Maranoa     17944     78554  0.2284
Blair       18078     71299  0.2536
Wide Bay    19052     74205  0.2567
Groom       21406     78067  0.2742
Gwydir      19274     69355  0.2779

### How predictiveis the geographic size of seat on support for the referendum?

#### Prepare data

In [33]:
# import geographic size of seats
filepath = '1999_referenda/electorate_boundaries/boundaries_republic_referendum_aus.csv'

df_area = pd.read_csv(
    filepath,
    skiprows = 1,
    names = ['electorate','area_sqkm']
)

# make df grouped by electorate

df_by_electorate = df_results[['electorate','yes_n','formal_n']].groupby('electorate').sum()
df_by_electorate['yes_p'] = round(r['yes_n']/r['formal_n'],4)

df_by_electorate = df_by_electorate.reset_index()

# merge in area
df_by_electorate = pd.merge(df_by_electorate, df_area, on='electorate', how='left')

display(df_by_electorate.head(5))

electorate  yes_n  formal_n   yes_p  area_sqkm
0   Adelaide  45580     80832  0.5639      80.80
1      Aston  43210     83822  0.5155     128.10
2   Ballarat  32784     80129  0.4091   11128.95
3      Banks  34719     75875  0.4576      61.70
4     Barker  26709     82364  0.3243   67077.85

#### create data for scatterplot

In [34]:
from plotly.offline import *
import plotly.offline as py
import plotly.plotly as pyonline
import plotly.graph_objs as go
init_notebook_mode(connected=True) # render plotly charts in the notebook on the fly

In [35]:
series = go.Scatter(
    y = df_by_electorate['yes_p'],
    x = df_by_electorate['area_sqkm'],
    name = '% Yes',
    mode = 'markers',
    text = df_by_electorate['electorate'],
    marker = dict (
        size = 10,
        opacity = 0.6
    )
)

xaxis=dict(
        title = 'Size of Electorate, SqKm',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
)

yaxis = dict(
        title = '% Support',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        tickformat = ',.0%',
        range=[.2,.8]
)

title = '1999 Republic Referendum - % Support vs. Size of Electorate'

titlefont = dict(
        family='Open Sans',
        size=22
)

layout = go.Layout(
    title = title,
    titlefont = titlefont,
    xaxis = xaxis,
    yaxis = yaxis
)

data = [series]

figure01 = go.Figure(data=data, layout=layout)

In [36]:
py.iplot(figure01, filename='figure01')
# pyonline.image.ishow(figure01, width=1500, height=750)

### Same chart, using a log scale for electorate size

In [37]:
xaxis=dict(
        title = 'Log of size of Electorate, SqKm',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        type='log'
)

title = '1999 Republic Referendum - % Support vs. log(size) of Electorate'

layout = go.Layout(
    title = title,
    titlefont = titlefont,
    xaxis = xaxis,
    yaxis = yaxis
)

figure02 = go.Figure(data=data, layout=layout)

There appears to be a relatively strong relationship between % support for the republic, and size of electorate

Smaller electorates by area (i.e, more densely populated inner-urban electorates) are more likely to support the republic

A noteable outlier is the Northern Terirtory, the second largest electorate by area, still had 49% support

In [38]:
py.iplot(figure02, filename='figure02')
# pyonline.image.ishow(figure02, width=1500, height=750)

### How much does the size of the seat predict support for the republic?

In [39]:
import math
import statsmodels.formula.api as sm

# add log(area) var to df
df_by_electorate['area_sqkm_log'] = df_by_electorate['area_sqkm'].apply(lambda x: math.log(x))
df_by_electorate.head(3)

# run regression
result = sm.ols(formula="area_sqkm_log ~ yes_p", data=df_by_electorate).fit()

display(result.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          area_sqkm_log   R-squared:                       0.560
Model:                            OLS   Adj. R-squared:                  0.557
Method:                 Least Squares   F-statistic:                     185.9
Date:                Wed, 02 Aug 2017   Prob (F-statistic):           8.03e-28
Time:                        10:33:22   Log-Likelihood:                -301.06
No. Observations:                 148   AIC:                             606.1
Df Residuals:                     146   BIC:                             612.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     16.4038      0.703     23.338      0.000      15.015      17.793
yes_p        -20.8541      1.529    -13.635      0.000     -23.877     -17.831
==============================================================================
Omnibus:                       26.282   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               41.692
Skew:                           0.893   Prob(JB):                     8.84e-10
Kurtosis:                       4.889   Cond. No.                         12.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### You can explain 56% of the variation in support for the Republic with seat size

### Yes vote v size of polling place

In [40]:
series = go.Scatter(
    y = df_results_by_pp['yes_p'],
    x = df_results_by_pp['total_n'],
    name = '% Yes',
    mode = 'markers',
    text = df_results_by_pp['polling_place'],
    marker = dict (
        size = 10,
        opacity = 0.6
    )
)

xaxis=dict(
        title = 'Number of Votes',
        titlefont=dict(
            family='Open Sans',
            size=16
        )
)

yaxis = dict(
        title = '% Support',
        titlefont=dict(
            family='Open Sans',
            size=16
        ),
        tickformat = ',.0%'
)

title = '1999 Republic Referendum - % Support vs. Number of Votes by polling place'

titlefont = dict(
        family='Open Sans',
        size=22
)

layout = go.Layout(
    title = title,
    titlefont = titlefont,
    xaxis = xaxis,
    yaxis = yaxis
)

data = [series]

figure03 = go.Figure(data=data, layout=layout)

It's exceptionally noisy, but there is some relationship between % support for the republic a size of polling place

In [41]:
py.iplot(figure03, filename='figure03')
# pyonline.image.ishow(figure03, width=1500, height=750)

### Just for fun - is the republic referendum at all predictive of the 2016 federal election result?

In [42]:
import fiona
import geopandas as gp
from shapely.geometry import Point
%matplotlib inline

df_pp['geometry'] = df_pp.apply(lambda z: Point(z.longitude, z.latitude), axis=1)

df_pp_geom = gp.GeoDataFrame(df_pp)

df_pp_geom.head(3)

state      polling_place                 premises  \
0   ACT           Bonython  Bonython Primary School   
1   ACT            Calwell      Calwell High School   
2   ACT  Canberra Hospital    The Canberra Hospital   

                     address    suburb  postcode wheelchair_access  \
0                 Hurtle Ave  BONYTHON    2905.0                 F   
1                 Casey Cres   CALWELL    2905.0                 F   
2  Blding 2 Level 3 Yamba Dr    GARRAN    2605.0                 F   

          match_source                            match_type  latitude  \
0  2007 Polling Places  Match 01 - state, premises, postcode  -35.4318   
1  2007 Polling Places  Match 01 - state, premises, postcode  -35.4406   
2  2007 Polling Places  Match 01 - state, premises, postcode  -35.3453   

   longitude                  geometry  
0    149.083  POINT (149.083 -35.4318)  
1    149.116  POINT (149.116 -35.4406)  
2    149.100    POINT (149.1 -35.3453)

In [43]:
# display(df_pp_geom.plot())

#### 2016 polling places

In [58]:
filepath = '2016_federal_election_data/polling_places_2016.csv'
df_pp_2016 = pd.read_csv(
    filepath
)

display(df_pp_2016.head(3))

State  DivisionID DivisionNm  PollingPlaceID  PollingPlaceTypeID  \
0   ACT         101   Canberra            8829                   1   
1   ACT         101   Canberra           64583                   5   
2   ACT         101   Canberra           65504                   5   

            PollingPlaceNm                  PremisesNm      PremisesAddress1  \
0                   Barton         Telopea Park School  New South Wales Cres   
1  Belconnen CANBERRA PPVC  Belconnen Community Centre        26 Chandler St   
2        BLV Canberra PPVC                BLV Canberra   50 Marcus Clarke St   

  PremisesAddress2 PremisesAddress3 PremisesSuburb PremisesStateAb  \
0              NaN              NaN         BARTON             ACT   
1              NaN              NaN      BELCONNEN             ACT   
2              NaN              NaN  CANBERRA CITY             ACT   

   PremisesPostCode   Latitude   Longitude  
0            2600.0 -35.315100  149.135000  
1            2900.0 -35.238904  149.070109  
2            2601.0 -35.279054  149.126379

In [45]:
def left_of_bracket(s):
    if '(' in s:
        needle = s.find('(')
        r = s[:needle-1].strip()
        return r
    else:
        return s

lambda_polling_places = lambda x: left_of_bracket(x)

# just ordinary polling places
df_pp_2016 = df_pp_2016[df_pp_2016['PollingPlaceTypeID'] == 1]

# make a seat-independent polling place column
df_pp_2016['polling_place'] = df_pp_2016['PollingPlaceNm'].apply(lambda_polling_places)

# filter for relevant columns
df_pp_2016 = df_pp_2016[[
    'State',
    'polling_place',
    'Latitude',
    'Longitude'
]]

# make headers lower case
df_pp_2016.columns = [x.lower() for x in df_pp_2016.columns]

# de dup
df_pp_2016.drop_duplicates()

df_pp_2016.head(3)

df_pp_2016.to_csv(
    '2016_federal_election_data/polling_places_2016_ordinary.csv'
)

In [46]:
df_pp_2016['geometry'] = df_pp_2016.apply(lambda z: Point(z.longitude, z.latitude), axis=1)

df_pp_2016_geom = gp.GeoDataFrame(df_pp_2016)

df_pp_2016_geom.head(3)

state polling_place  latitude  longitude                     geometry
0   ACT        Barton -35.31510  149.13500     POINT (149.135 -35.3151)
3   ACT      Bonython -35.43180  149.08300     POINT (149.083 -35.4318)
4   ACT       Braddon -35.27363  149.14001  POINT (149.14001 -35.27363)

In [47]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points

# makes geometry points for each pp
pts = df_pp_2016_geom.geometry.unary_union

# for a given point return nearest poling place
def near(point, polling_places=pts):

    # get the data point from df_pp_2016_geom for which geometry = the geometry of the nearest point
    nearest = df_pp_2016_geom.geometry == nearest_points(point,polling_places)[1]
    # return the index col of pp_2016
    return df_pp_2016_geom[nearest].index.get_values()[0]

# test run, limit dataset
df_pp_geom = df_pp_geom.head(50)

# run 'near' into a new column on the 1999 data frame
df_pp_geom['pp_2016_index'] = df_pp_geom.apply(lambda row: near(row.geometry), axis=1)

display(df_pp_geom.head(3))

state      polling_place                 premises  \
0   ACT           Bonython  Bonython Primary School   
1   ACT            Calwell      Calwell High School   
2   ACT  Canberra Hospital    The Canberra Hospital   

                     address    suburb  postcode wheelchair_access  \
0                 Hurtle Ave  BONYTHON    2905.0                 F   
1                 Casey Cres   CALWELL    2905.0                 F   
2  Blding 2 Level 3 Yamba Dr    GARRAN    2605.0                 F   

          match_source                            match_type  latitude  \
0  2007 Polling Places  Match 01 - state, premises, postcode  -35.4318   
1  2007 Polling Places  Match 01 - state, premises, postcode  -35.4406   
2  2007 Polling Places  Match 01 - state, premises, postcode  -35.3453   

   longitude                  geometry  pp_2016_index  
0    149.083  POINT (149.083 -35.4318)              3  
1    149.116  POINT (149.116 -35.4406)              5  
2    149.100    POINT (149.1 -35.3453)             19

In [51]:
# output to csv
# commented out to prevent exporting by accident - the above code takes a while to run
df_pp_geom.to_csv('1999_referenda_output/polling_places_with_nearest_2016_polling_place.csv',index=False)

### Import Nearest 2016 polling place to each 1999 polling place data

This is done here so we don't have to rerun the matching code block each execution, as it is quite slow to run

In [52]:
filepath = '1999_referenda_output/polling_places_with_nearest_2016_polling_place.csv'
df_pp_1999_nearest_2016 = pd.read_csv(
    filepath
)
df_pp_1999_nearest_2016.head(3)

state      polling_place                 premises  \
0   ACT           Bonython  Bonython Primary School   
1   ACT            Calwell      Calwell High School   
2   ACT  Canberra Hospital    The Canberra Hospital   

                     address    suburb  postcode wheelchair_access  \
0                 Hurtle Ave  BONYTHON    2905.0                 F   
1                 Casey Cres   CALWELL    2905.0                 F   
2  Blding 2 Level 3 Yamba Dr    GARRAN    2605.0                 F   

          match_source                            match_type  latitude  \
0  2007 Polling Places  Match 01 - state, premises, postcode  -35.4318   
1  2007 Polling Places  Match 01 - state, premises, postcode  -35.4406   
2  2007 Polling Places  Match 01 - state, premises, postcode  -35.3453   

   longitude                  geometry  pp_2016_index  
0    149.083  POINT (149.083 -35.4318)              3  
1    149.116  POINT (149.116 -35.4406)              5  
2    149.100    POINT (149.1 -35.3453)             19

### Merge in 2016 swing, republic support

In [81]:
# import swing data

filepath = '2016_federal_election_data/two_party_preferred_by_polling_place_2016.csv'
df_sw_2016 = pd.read_csv(
    filepath
)

display(df_sw_2016.head(3))

# tidy up 2016 swing data

# make headers lower case
df_sw_2016.columns = [x.lower() for x in df_sw_2016.columns]

# make a seat-independent polling place column
lambda_polling_places = lambda x:left_of_bracket(x)
df_sw_2016['polling_place'] = df_sw_2016['pollingplace'].apply(lambda_polling_places)

# create a label for state
df_sw_2016['state'] = df_sw_2016['stateab']

# filter for relevant columns
df_sw_2016 = df_sw_2016[[
    'state',
    'polling_place',
    'swing',
    'totalvotes'
]]

# convert swing to a percentage
df_sw_2016['swing'] = df_sw_2016['swing']/100

# before merge
print("Before Merge:")
display(df_sw_2016[df_sw_2016['polling_place']=="Braddon"])

# make a weighted swing column
df_sw_2016['weight'] = df_sw_2016['swing'] * df_sw_2016['totalvotes']

print("With weight:")
display(df_sw_2016[df_sw_2016['polling_place']=="Braddon"])

del df_sw_2016['swing']

df_sw_2016 = df_sw_2016.groupby(['state','polling_place']).agg('sum')
df_sw_2016 = df_sw_2016.reset_index()
df_sw_2016['swing'] = df_sw_2016['weight']/df_sw_2016['totalvotes']

print("Merged with weight:")
display(df_sw_2016[df_sw_2016['polling_place']=="Braddon"])

del df_sw_2016['weight']

# after merge
print("Final:")
display(df_sw_2016[df_sw_2016['polling_place']=="Braddon"])

display(df_sw_2016.head(5))


StateAb  DivisionID DivisionNm  PollingPlaceID             PollingPlace  \
0     ACT         101   Canberra            8829                   Barton   
1     ACT         101   Canberra           64583  Belconnen CANBERRA PPVC   
2     ACT         101   Canberra           65504        BLV Canberra PPVC   

   Liberal/National Coalition Votes  Liberal/National Coalition Percentage  \
0                               991                                  44.40   
1                               446                                  39.86   
2                                16                                  48.48   

   Australian Labor Party Votes  Australian Labor Party Percentage  \
0                          1241                              55.60   
1                           673                              60.14   
2                            17                              51.52   

   TotalVotes  Swing  
0        2232   2.19  
1        1119   0.57  
2          33  15.15

Before Merge:


state polling_place   swing  totalvotes
4    ACT       Braddon  0.0517        1249
65   ACT       Braddon -0.0176        1457

With weight:


state polling_place   swing  totalvotes   weight
4    ACT       Braddon  0.0517        1249  64.5733
65   ACT       Braddon -0.0176        1457 -25.6432

Merged with weight:


state polling_place  totalvotes   weight     swing
11   ACT       Braddon        2706  38.9301  0.014387

Final:


state polling_place  totalvotes     swing
11   ACT       Braddon        2706  0.014387

state      polling_place  totalvotes   swing
0   ACT      Ainslie North        2257 -0.0010
1   ACT             Amaroo        2724 -0.0325
2   ACT             Aranda        2447  0.0070
3   ACT  BLV Canberra PPVC          33  0.1515
4   ACT    BLV Fenner PPVC          24 -0.1576